In [34]:
import sagemaker

bucket = 'plansoft-sagemaker-mask-detect'
prefix = ''
data_directory = 'data/training'
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::960704241809:role/plansoft-research-sagemaker


In [35]:
inputs = 's3://' + bucket + '/' + prefix + data_directory
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://plansoft-sagemaker-mask-detect/data/training


In [36]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                role=role,
                framework_version='1.6.0',
                train_instance_count=1,
                train_instance_type='ml.m5.large',
                # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                #                            bptt, dropout, tied, seed, log_interval
                hyperparameters={
                    'epochs': 1
                })

In [37]:
estimator.fit({'training': inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-15 09:28:16 Starting - Starting the training job...
2020-10-15 09:28:18 Starting - Launching requested ML instances......
2020-10-15 09:29:19 Starting - Preparing the instances for training...
2020-10-15 09:30:00 Downloading - Downloading input data......
2020-10-15 09:31:06 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-15 09:31:07,400 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-15 09:31:07,402 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-15 09:31:07,412 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-15 09:31:13,652 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-15 09:31:13,938 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus insta

UnexpectedStatusException: Error for Training job pytorch-training-2020-10-15-09-28-15-860: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python train.py --epochs 1"
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
  0%|          | 0.00/160M [00:00<?, ?B/s]  1%|â         | 2.28M/160M [00:00<00:06, 23.8MB/s]  3%|â         | 4.44M/160M [00:00<00:06, 23.4MB/s]  4%|â         | 6.88M/160M [00:00<00:06, 24.0MB/s]  6%|â         | 9.31M/160M [00:00<00:06, 24.3MB/s]  7%|â         | 11.1M/160M [00:00<00:07, 20.9MB/s]  8%|â         | 13.6M/160M [00:00<00:06, 22.1MB/s] 10%|â         | 15.5M/160M [00:00<00:07, 21.4MB/s] 11%|ââ        | 18.1M/160M [00:00<00:06, 23.0MB/s] 13%|ââ        | 20.3M/160M [00:00<00:07, 20.4MB/s] 14%|ââ        | 22.9M/160M [00:01<00:06, 22.1MB/s] 16%|ââ        | 25.6M/160M [00:0

In [ ]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [ ]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.6.0',
                     entry_point='generate.py',
                     predictor_cls=JSONPredictor)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
# input = {
#     'seed': 111,
#     'temperature': 2.0,
#     'words': 100
# }
# response = predictor.predict(input)
# print(response)

In [ ]:
# sagemaker_session.delete_endpoint(predictor.endpoint)